In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('csv_files/sap_produkcyjne_przyklad.csv', encoding='utf-8-sig', sep=";")

In [3]:
with open('csv_files/sap_produkcyjne_przyklad.csv', encoding="utf-8") as f:
    first_line = f.readline()

import re
od_match = re.search(r"Od\s+(\d{2}\.\d{2}\.\d{4})", first_line)
do_match = re.search(r"Do\s+(\d{2}\.\d{2}\.\d{4})", first_line)

od_date = pd.to_datetime(od_match.group(1), dayfirst=True).date() if od_match else None
do_date = pd.to_datetime(do_match.group(1), dayfirst=True).date() if do_match else None

In [4]:
header_row_1 = df.iloc[2].fillna("").astype(str).str.strip()
header_row_2 = df.iloc[3].fillna("").astype(str).str.strip()
combined_headers = [
    (a + " " + b).strip().replace(".", "").replace("/", "").replace(" ", "")
    for a, b in zip(header_row_1, header_row_2)
]

df = df.iloc[5:].copy()
df.columns = combined_headers
df.dropna(how="all", inplace=True)
df.reset_index(drop=True, inplace=True)

In [5]:
df["Od"] = od_date
df["Do"] = do_date

In [6]:
df.columns = [col.replace('%', 'Procent') for col in df.columns]
print(df.columns)

Index(['Linia', 'Rodzina', 'QLTOTAkt', 'QLTOTPln', 'ProcentDvtProduk',
       'ZmianaCzysty', 'ZmianaPrg', 'ZmianaStd', 'QZmStandard', 'QZmStd',
       'QZmAkt', 'QZmDocel', 'QZmStd', 'QCPKAkt', 'QCPKDocel', 'QCPKStd',
       'OpeLNShAkt', 'OpeLNShDocel', 'OpeLNShStd', 'OpeELShAkt',
       'OpeELShDocel', 'OpeELShStd', 'GQLAkt', 'GQLDocel', 'GQLStd',
       'ProcentSCEff', 'ProcentSCStd', 'ProcentSCStd', 'ProcentSREff',
       'ProcentSRStd', 'ProcentSRStd', 'ProcentSFSPEff', 'ProcentSFSPStd',
       'ProcentSFSPStd', 'GodzPracAkt', 'GodzPracDocel', 'GodzPracStd',
       'ProcentELINIAEff', 'ProcentELINIAObb', 'ProcentELINIAStd',
       'ProcentEPracEff', 'ProcentEPracObb', 'ProcentEPracStd',
       'ProcentZyskuEff', 'ProcentZyskuObb', 'ProcentZyskuStd', 'Od', 'Do'],
      dtype='object')


In [7]:
float_columns = [
    "QLTOTAkt", "QLTOTPln", "ProcentDvtProduk", "ZmianaCzysty", "ZmianaPrg", "ZmianaStd",
    "QZmAkt", "QZmDocel", "QZmStd", "QCPKAkt", "QCPKDocel", "QCPKStd",
    "OpeLNShAkt", "OpeLNShDocel", "OpeLNShStd", "OpeELShAkt", "OpeELShDocel", "OpeELShStd",
    "GQLAkt", "GQLDocel", "GQLStd", "ProcentSCEff", "ProcentSCStd", "ProcentSREff", "ProcentSRStd",
    "ProcentSFSPEff", "ProcentSFSPStd", "GodzPracAkt", "GodzPracDocel", "GodzPracStd",
    "ProcentELINIAEff", "ProcentELINIAObb", "ProcentELINIAStd",
    "ProcentEPracEff", "ProcentEPracObb", "ProcentEPracStd",
    "ProcentZyskuEff", "ProcentZyskuObb", "ProcentZyskuStd"
]

df = df.loc[:, ~df.columns.duplicated()]
df.drop(columns=["QZmStandard"], inplace=True)
for col in float_columns:
    # Access column safely as a Series
    df.loc[:, col] = df[col].apply(lambda x: str(x).replace(' ', '').replace(',', '.'))
    df.loc[:, col] = pd.to_numeric(df[col], errors='coerce')

In [8]:
df = df.astype({
    "Od": "datetime64[ns]",
    "Do": "datetime64[ns]",
    "Linia": "int",
    "Rodzina": "string",
    "QLTOTAkt": "float",
    "QLTOTPln": "float",
    "ProcentDvtProduk": "float",
    "ZmianaCzysty": "float",
    "ZmianaPrg": "float",
    "ZmianaStd": "float",
    "QZmAkt": "float",
    "QZmDocel": "float",
    "QZmStd": "float",
    "QCPKAkt": "float",
    "QCPKDocel": "float",
    "QCPKStd": "float",
    "OpeLNShAkt": "float",
    "OpeLNShDocel": "float",
    "OpeLNShStd": "float",
    "OpeELShAkt": "float",
    "OpeELShDocel": "float",
    "OpeELShStd": "float",
    "GQLAkt": "float",
    "GQLDocel": "float",
    "GQLStd": "float",
    "ProcentSCEff": "float",
    "ProcentSCStd": "float",
    "ProcentSREff": "float",
    "ProcentSRStd": "float",
    "ProcentSFSPEff": "float",
    "ProcentSFSPStd": "float",
    "GodzPracAkt": "float",
    "GodzPracDocel": "float",
    "GodzPracStd": "float",
    "ProcentELINIAEff": "float",
    "ProcentELINIAObb": "float",
    "ProcentELINIAStd": "float",
    "ProcentEPracEff": "float",
    "ProcentEPracObb": "float",
    "ProcentEPracStd": "float",
    "ProcentZyskuEff": "float",
    "ProcentZyskuObb": "float",
    "ProcentZyskuStd": "float"
})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 43 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Linia             1 non-null      int32         
 1   Rodzina           1 non-null      string        
 2   QLTOTAkt          1 non-null      float64       
 3   QLTOTPln          1 non-null      float64       
 4   ProcentDvtProduk  1 non-null      float64       
 5   ZmianaCzysty      1 non-null      float64       
 6   ZmianaPrg         1 non-null      float64       
 7   ZmianaStd         1 non-null      float64       
 8   QZmStd            1 non-null      float64       
 9   QZmAkt            1 non-null      float64       
 10  QZmDocel          1 non-null      float64       
 11  QCPKAkt           1 non-null      float64       
 12  QCPKDocel         1 non-null      float64       
 13  QCPKStd           1 non-null      float64       
 14  OpeLNShAkt        1 non-null  

In [9]:
from sqlalchemy import create_engine

# Connection details
server = 'JASNACZERN'        
database = 'ferrero'
username = 'sa'
password = '1234'
driver = 'ODBC Driver 17 for SQL Server'  

# SQLAlchemy connection string
conn_str = (
    f"mssql+pyodbc://{username}:{password}@{server}/{database}"
    f"?driver={driver.replace(' ', '+')}"
)

# Create SQLAlchemy engine
engine = create_engine(conn_str)

# Write the DataFrame to SQL Server (replace table_name with your desired name)
df.to_sql('BilansProdukcji', con=engine, if_exists='append', index=False)

1